# Explore the dataset


In this notebook, we will perform an EDA (Exploratory Data Analysis) on the processed Waymo dataset (data in the `processed` folder). In the first part, you will create a function to display 

In [20]:
from utils import get_dataset
import glob
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import numpy as np
import tensorflow as tf

%matplotlib inline


In [21]:
!pwd

dataset = get_dataset("/data/**/*.tfrecord")
dataset

/home/workspace
INFO:tensorflow:Reading unweighted datasets: ['/data/**/*.tfrecord']
INFO:tensorflow:Reading record datasets for input file: ['/data/**/*.tfrecord']
INFO:tensorflow:Number of filenames to read: 103


<DatasetV1Adapter shapes: {image: (None, None, 3), source_id: (), key: (), filename: (), groundtruth_image_confidences: (None,), groundtruth_verified_neg_classes: (None,), groundtruth_not_exhaustive_classes: (None,), groundtruth_boxes: (None, 4), groundtruth_area: (None,), groundtruth_is_crowd: (None,), groundtruth_difficult: (None,), groundtruth_group_of: (None,), groundtruth_weights: (None,), groundtruth_classes: (None,), groundtruth_image_classes: (None,), original_image_spatial_shape: (2,)}, types: {image: tf.uint8, source_id: tf.string, key: tf.string, filename: tf.string, groundtruth_image_confidences: tf.float32, groundtruth_verified_neg_classes: tf.int64, groundtruth_not_exhaustive_classes: tf.int64, groundtruth_boxes: tf.float32, groundtruth_area: tf.float32, groundtruth_is_crowd: tf.bool, groundtruth_difficult: tf.int64, groundtruth_group_of: tf.bool, groundtruth_weights: tf.float32, groundtruth_classes: tf.int64, groundtruth_image_classes: tf.int64, original_image_spatial_sh

## Write a function to display an image and the bounding boxes

Implement the `display_instances` function below. This function takes a batch as an input and display an image with its corresponding bounding boxes. The only requirement is that the classes should be color coded (eg, vehicles in red, pedestrians in blue, cyclist in green).

In [23]:
def display_images(batch):
    try:
        
        obj_dict = {1: 'red', 2: 'blue', 4: 'green'}
    
    
        fig, ax = plt.subplots() 
        im = batch["image"].numpy()
        ax.imshow(im)
        #coordinates=c, label=l,
        #y1,x1,y2,x2
        for c, l in zip(batch["groundtruth_boxes"].numpy(), batch["groundtruth_classes"].numpy()):
           # xy = (c[1]*im.shape[1], c[0]*im.shape[0])
            x=c[1]*im.shape[1]
            y=c[0]*im.shape[0]
            width = (c[3] - c[1])*im.shape[1]
            height = (c[2] - c[0])*im.shape[0]
            rec = patches.Rectangle((x,y), width, height, linewidth=1, edgecolor=obj_dict[l], facecolor='none')
            ax.add_patch(rec)
        plt.axis("off")
    except Exception as err:
        print(err)

## Display 10 images 

Using the dataset created in the second cell and the function you just coded, display 10 random images with the associated bounding boxes. You can use the methods `take` and `shuffle` on the dataset.

In [ ]:
# Display 10 random images in dataset
plt.figure(figsize=(10, 10))
for i in range(10):
    for batch in dataset.shuffle(10, reshuffle_each_iteration=True).take(1):
        display_images(batch)
plt.show()

## Additional EDA

In this last part, you are free to perform any additional analysis of the dataset. What else would like to know about the data?
For example, think about data distribution. So far, you have only looked at a single file...

In [ ]:
objects={1:0,2:0,4:0}

for batches in dataset.take(1000):
    for label in batches['groundtruth_classes'].numpy():
        objects[label]=objects[label]+1
print(objects)

In [ ]:
cars=objects[1]
pedestrians=objects[2]
cyclists=objects[4]

detectedObject=['Car','Pedestrian','Cyclist']
foundClasses=[cars,pedestrians,cyclists]

colors=['red','blue','green']

plt.bar(detectedObject,foundClasses,color=colors)
plt.title("Distribution of found Classes in a 1000 random images",fontsize=20)
plt.xlabel('Class name',fontsize=16)
plt.ylabel('Number of classes detected',fontsize=16)
plt.show()